In [ ]:
%pip install langchain
%pip install langchain-deepseek
%pip install langchain-ollama
%pip install python-dotenv

In [ ]:

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env",override=True)
from os import getenv

In [ ]:
from langchain_deepseek import ChatDeepSeek

key = getenv("DEEPSEEK_API_KEY") 
url = getenv("DEEPSEEK_API_BASE")
model = getenv("DEEPSEEK_MODEL")

# print(key, url, model)

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:latest",
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg)

In [ ]:
import os

key = getenv("SF_API_KEY") 
url = getenv("SF_API_BASE")
model = getenv("SF_MODEL")

os.environ["DEEPSEEK_API_KEY"] = key
os.environ["DEEPSEEK_API_BASE"] = url
print(key, url, model)

In [ ]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)
messages = [
    SystemMessage("Translate the following from English into Chinese."),
    HumanMessage("hi!"),
]

print(llm.invoke(messages).content)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

system_template = "Translate the following from English into {language}"
human_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), 
     ("user", human_template)]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))


In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

system_template = SystemMessagePromptTemplate.from_template( "Translate the following from English into {language}")
human_template = HumanMessagePromptTemplate.from_template("{text}")

prompt_template = ChatPromptTemplate.from_messages(
    [system_template, 
     human_template]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)
input_data = "What is the capital of France?"
summary_chain = (
    ChatPromptTemplate.from_template("Summarize this: {input}")
    | llm
    | StrOutputParser()
)
sentiment_chain = (
    ChatPromptTemplate.from_template("What is the sentiment of this: {input}")
    | llm
    | StrOutputParser()
)

parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})
parallel_result = parallel_chain.invoke({"input": input_data})
print(parallel_result)

In [12]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

# 定义子链
tech_prompt = ChatPromptTemplate.from_template(
    "作为科技作者，用专业术语解释：{concept}"
)
sports_prompt = ChatPromptTemplate.from_template(
    "作为体育解说员，用生动语言描述：{concept}"
)
general_prompt = ChatPromptTemplate.from_template(
    "用通俗语言解释：{concept}"
)

tech_chain = tech_prompt | llm | StrOutputParser()
sports_chain = sports_prompt | llm | StrOutputParser()
general_chain = general_prompt | llm | StrOutputParser()

# 构建分支逻辑
branch_chain = RunnableBranch(
    (lambda x: x["topic"] == "科技", tech_chain),
    (lambda x: x["topic"] == "体育", sports_chain),
    general_chain
)

# 完整执行链
# full_chain = RunnableParallel({  # 并行处理输入
#     "concept": lambda x: x["concept"],
#     "topic": lambda x: x["topic"]
# }) | branch_chain
full_chain = branch_chain

print("***********************")
print(full_chain.invoke({
    "concept": "神经网络", 
    "topic": "科技"
}))
print("***********************")

print(full_chain.invoke({
    "concept": "越位", 
    "topic": "体育"
}))


各位观众朋友们，让我们来聊聊足球场上那个让人又爱又恨的规则——越位！

想象一下，比赛正进行到白热化阶段，前锋如同一只猎豹，突然加速，甩开防守球员，接到队友的精准传球，一脚劲射破门！全场沸腾，球迷们欢呼雀跃！然而，就在这激动人心的时刻，边裁举起了手中的旗帜，主裁判也吹响了哨声——**越位！进球无效！**

越位，这个规则就像足球场上的“隐形裁判”，时刻盯着球员们的一举一动。简单来说，当进攻球员在接球的瞬间，**身体的有效部位（通常是脚或躯干）**比倒数第二名防守球员（通常是最后一名后卫）更靠近对方球门，并且处于对方半场，那么他就越位了。除非，他接球时在自己的半场，或者他接球时与倒数第二名防守球员平行。

越位的判罚往往伴随着争议，因为它需要裁判在电光火石之间做出判断。有时，球员只是脚尖越过了那条“隐形线”，进球就被无情地取消；有时，裁判的视线被遮挡，误判也会发生。但正是这种精确到毫厘的规则，让足球比赛充满了悬念和戏剧性。

越位就像一场“猫鼠游戏”，前锋们总是试图在最后一刻“偷跑”，而防守球员则拼命保持阵型，不让对手有机可乘。每一次越位判罚，都可能改变比赛的走向，甚至决定胜负。

所以，下次当你看到边裁举旗，听到裁判哨声，别急着失望或欢呼，先看看是不是越位！这就是足球，规则与激情并存，细节决定成败！
